In [ ]:
import string

import numpy as np

from nltk.tokenize import WordPunctTokenizer

from matplotlib import pyplot as plt

from IPython.display import clear_output

Рассмотрим корпус вопросов с сайта Quora.

In [ ]:
# download the data:
#!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt -nc

!wget https://raw.githubusercontent.com/MSUcourses/Data-Analysis-with-Python/main/Deep%20Learning/Files/quora.txt -O ./quora.txt -nc
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

--2024-03-13 11:05:57--  https://raw.githubusercontent.com/MSUcourses/Data-Analysis-with-Python/main/Deep%20Learning/Files/quora.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33813915 (32M) [text/plain]
Saving to: ‘./quora.txt’

./quora.txt         100%[===================>]  32.25M   165MB/s    in 0.2s    

2024-03-13 11:05:59 (165 MB/s) - ‘./quora.txt’ saved [33813915/33813915]



Посмотрим на элемент данных:

In [ ]:
data = list(open("./quora.txt", encoding="utf-8"))                                #вопрос №55 из quora
data[55]

'What are all the pros and cons of having dual citizenship?\n'

Произведем токенизацию и базовую предобработку:

In [ ]:
tokenizer = WordPunctTokenizer()                                               # бьет предложение на слова и знаки препинания

print(tokenizer.tokenize(data[55]))
data_tok = [tokenizer.tokenize(x.lower()) for x in data]

['What', 'are', 'all', 'the', 'pros', 'and', 'cons', 'of', 'having', 'dual', 'citizenship', '?']


Для начала, обучим word2vec на доступном наборе данных. Строить для этого модель вручную не понадобится, она уже доступна в `gensim`.

In [ ]:
from gensim.models import Word2Vec
model_obj = Word2Vec(data_tok,
                 vector_size=32, #для каждого слова строится 32-мерный вектор
                 min_count=5,
                 window=5)
model = model_obj.wv #сохраняем соответствие слов и их векторов в переменную model

Теперь нам доступны векторы для любого слова из словаря:

In [ ]:
model.get_vector('cat')

array([-3.5631819 ,  1.0918669 , -0.8742508 , -0.99347484, -1.430906  ,
        0.6366659 , -1.7592809 , -2.675177  , -1.3768494 ,  2.665527  ,
        1.5751318 , -2.682338  , -1.4218243 ,  1.6961786 ,  2.371728  ,
       -3.617359  , -2.9813735 ,  0.11653395, -1.9565647 , -1.3708756 ,
        1.8075705 ,  1.6682189 , -1.5121067 , -0.03990271,  0.690471  ,
        1.6506522 , -2.275169  , -0.45736673, -0.7651789 ,  1.524998  ,
        2.0204737 , -0.69962317], dtype=float32)

Так как слова представлены векторами, теперь можно вычислить расстояние (или некоторую меру схожести) между ними. Например, можно оценить, какие слова наиболее близки к заданному.

In [ ]:
model.most_similar('parent') #используется косинусная мера близости

[('child', 0.9181265234947205),
 ('affair', 0.8672084212303162),
 ('mother', 0.8464205861091614),
 ('sibling', 0.8411386609077454),
 ('kid', 0.829600989818573),
 ('lover', 0.8295760154724121),
 ('partner', 0.8194788098335266),
 ('father', 0.8181169629096985),
 ('siblings', 0.8100043535232544),
 ('wife', 0.8080825209617615)]

Загрузим предобученные эмбеддинги размерности 25. Они были обучены на данных из Twitter.

In [ ]:
import gensim.downloader as api
model = api.load('glove-twitter-25') # здесь используются векторы GloVe

[==================================================] 100.0% 104.8/104.8MB downloaded


In [ ]:
model.most_similar(positive=["лето"])

[('утро', 0.8916537761688232),
 ('весна', 0.878777265548706),
 ('зима', 0.8693447113037109),
 ('скоро', 0.8552830219268799),
 ('солнце', 0.8396058082580566),
 ('ночь', 0.8204128742218018),
 ('надеюсь', 0.8200833797454834),
 ('воскресенье', 0.8176007270812988),
 ('летом', 0.8077778816223145),
 ('солнышко', 0.8064454793930054)]

#### Визуализация векторных представлений слов

В данный момент каждое слово представлено вектором размерности 25. Для визуализации слов воспользуемся PCA.

In [ ]:
model.sort_by_descending_frequency() # выберем наиболее часто встречающиеся слова

In [ ]:
words = list(model.key_to_index.keys())[:1000] # берем топ-1000 самых встречающихся слов

print(words[::100])

word_vectors = np.asarray([model[x] for x in words])

['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei']


In [ ]:
# построим двумерное представление
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
pca = PCA(2)

scaler = StandardScaler()

word_vectors_pca = scaler.fit_transform(word_vectors)
word_vectors_pca = pca.fit_transform(word_vectors_pca)

In [ ]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x,
                                       'y' : y,
                                       'color': color, **kwargs })                # ColumnDataSource все по колонкам


    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color',
                alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

In [ ]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words) # можно увидеть, что слова разбились по языкам на несколько класеров

figure(id='p1004', ...)

Как видим, образовалось несколько кластеров.

#### Снижение размерности с помощью UMAP
Применим технику UMAP, которая учитывает соседей заданных точек.

In [ ]:
! pip install umap-learn
import umap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 5.8 MB/s eta 0:00:00
  Created wheel for umap-learn: filename=umap_learn-0.5.5-py3-none-any.whl size=86832 sha256=03f1d97b4f6d5aa79664a7e664f60969e264c6084811b7a82d949249f9e4b2af
  Stored in directory: /root/.cache/pip/wheels/3a/70/07/428d2b58660a1a3b431db59b806a10da736612ebbc66c1bcc5
Successfully built umap-learn


In [ ]:
embedding = umap.UMAP(n_neighbors=5).fit_transform(word_vectors) # 5 соседей, fit_transform(word_vectors) - преобразовали все исходные векторы

In [ ]:
draw_vectors(embedding[:, 0], embedding[:, 1], token=words)

figure(id='p1053', ...)

Кластеры стали более ярко выраженными.